In [1]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

data_version = "TMD_cov"
output_version = "20240910"

outdir = "/media/hieunguyen/HNSD_mini/outdir"
PROJECT = "TMD450_TCGA_data_analysis"
thres_hypo = 0.3
thres_hyper = 0.6

input_cancer_class = "Liver"

path_to_main_output = os.path.join(outdir, PROJECT, output_version)
path_to_07_output = os.path.join(outdir, PROJECT, output_version, input_cancer_class, "thres_hypo_{}_hyper_{}".format(thres_hypo, thres_hyper))

all_read_files = [item for item in pathlib.Path(os.path.join(path_to_07_output)).glob("*.csv")]
print("There are {} samples in the case {} vs control".format(len(all_read_files), input_cancer_class))

TypeError: 'PosixPath' object is not iterable